In [1]:
from pycocotools.coco import COCO
import tensorflow as tf
from model.training_config import TrainingConfig
from coco.coco import load_coco_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt

# physical_devices = tf.config.list_physical_devices('GPU')
# if len(physical_devices) == 1:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
config = TrainingConfig()

valid_dt, class_names = load_coco_dataset(
    config, 1, augmentation=True, img_dir="coco/val2017", ann_file="coco/annotations/instances_val2017.json")

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


In [3]:
for img,a,b in valid_dt:
    print(img.shape) # img
    print(a.shape) # box 
    print(b.shape) # class
    break

(224, 224, 3)
(1, 4)
(1, 1)


In [4]:
a[:,0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.67190623>

In [5]:
def loss(y_true, y_pred):
    # y_pred = (num,x1,x2,y1,y2,...)

    print(y_true)
    print(y_pred)

    num_loss = float(y_true[:,0,0][0] - y_pred[:,0,0][0]) ** 2
    iou_loss = 0
    print(num_loss)

    # for i in range(batch_size):
    #     num_true = float(y_true[i][0][0])
    #     num_pred = float(y_pred[i][0][0])
    #     num_loss += (num_true - num_pred) ** 2
        
        
    #     for j in range(num_true-1):
    #         print(y_true[i])
    #         print(y_pred[i])
    #         iou_loss += bbox_iou(
    #             y_true[i][j+1][0],
    #             y_pred[i][j+1][0]
    #         )

    return num_loss + iou_loss

def bbox_iou(boxes_1, boxes_2):
    """
    calculate regression loss using iou
    :param boxes_1: boxes_1 shape is [x, y, w, h]
    :param boxes_2: boxes_2 shape is [x, y, w, h]
    :return:
    """
    # transform [x, y, w, h] to [x_min, y_min, x_max, y_max]
    boxes_1 = tf.concat([boxes_1[..., :2] - boxes_1[..., 2:] * 0.5,
                         boxes_1[..., :2] + boxes_1[..., 2:] * 0.5], axis=-1)
    boxes_2 = tf.concat([boxes_2[..., :2] - boxes_2[..., 2:] * 0.5,
                         boxes_2[..., :2] + boxes_2[..., 2:] * 0.5], axis=-1)
    boxes_1 = tf.concat([tf.minimum(boxes_1[..., :2], boxes_1[..., 2:]),
                         tf.maximum(boxes_1[..., :2], boxes_1[..., 2:])], axis=-1)
    boxes_2 = tf.concat([tf.minimum(boxes_2[..., :2], boxes_2[..., 2:]),
                         tf.maximum(boxes_2[..., :2], boxes_2[..., 2:])], axis=-1)

    # calculate area of boxes_1 boxes_2
    boxes_1_area = (boxes_1[..., 2] - boxes_1[..., 0]) * (boxes_1[..., 3] - boxes_1[..., 1])
    boxes_2_area = (boxes_2[..., 2] - boxes_2[..., 0]) * (boxes_2[..., 3] - boxes_2[..., 1])

    # calculate the two corners of the intersection
    left_up = tf.maximum(boxes_1[..., :2], boxes_2[..., :2])
    right_down = tf.minimum(boxes_1[..., 2:], boxes_2[..., 2:])

    # calculate area of intersection
    inter_section = tf.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]

    # calculate union area
    union_area = boxes_1_area + boxes_2_area - inter_area

    # calculate iou add epsilon in denominator to avoid dividing by 0
    iou = inter_area / (union_area + tf.keras.backend.epsilon())

    return iou

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers,models

backbone = ResNet50()

# (224, 224, 3)
input_layer = layers.Input(shape=(224, 224, 3))
x = backbone(input_layer)
# x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(400, activation='relu')(x)
output_layer = layers.Reshape(target_shape=(100,4))(x)

model = models.Model(
    inputs = input_layer,
    outputs = output_layer
)

optimizer = tf.keras.optimizers.SGD(lr=0.001)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['loss']
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense (Dense)               (None, 1024)              1025024   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 400)               410000    
                                                                 
 reshape (Reshape)           (None, 100, 4)            0         
                                                                 
Total params: 28,121,336
Trainable params: 28,068,216
Non-tra

C:\Python38\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [7]:
model.fit(valid_dt.batch(32), epochs=10)

Epoch 1/10
Tensor("loss/cond/Identity:0", dtype=float32)
Tensor("loss/cond/Identity_1:0", shape=(None, 100, 4), dtype=float32)
Tensor("loss/pow:0", dtype=float32)


ValueError: in user code:

    File "C:\Python38\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Python38\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Python38\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Python38\lib\site-packages\keras\engine\training.py", line 817, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Python38\lib\site-packages\keras\engine\compile_utils.py", line 439, in update_state
        self.build(y_pred, y_true)
    File "C:\Python38\lib\site-packages\keras\engine\compile_utils.py", line 359, in build
        self._metrics = tf.__internal__.nest.map_structure_up_to(y_pred, self._get_metric_objects,
    File "C:\Python38\lib\site-packages\ker